Copyright 2021 The TensorFlow Authors.
```
@title Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.
You may obtain a copy of the License at
https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
```

# Tiền xử lý văn bản với BERT

## Tổng quan

Tiền xử lý văn bản là quá trình chuyển đổi từ đầu-đến-cuối của văn bản thô thành đầu vào số nguyên của mô hình. Các mô hình xử lý ngôn ngữ tự nhiên (NLP) thường đi kèm với hàng trăm (nếu không phải hàng nghìn) dòng mã Python để tiền xử lý văn bản. Tiền xử lý văn bản thường là một thách thức cho các mô hình bởi vì:
* **Độ nghiêng phục vụ-huấn luyện**: Ngày càng khó khăn để đảm bảo rằng logic tiền xử lý của các đầu vào của mô hình là nhất quán ở tất cả các giai đoạn phát triển mô hình (chẳng hạn tiền huấn luyện, tinh chỉnh, đánh giá và suy luận). Sử dụng các siêu tham số khác nhau, token hoá, các thuật toán tiền xử lý chuỗi hoặc đơn giản là đóng gói các đầu vào mô hình không nhất quán ở các giai đoạn khác nhau có thể mang lại lỗi khó gỡ và các tác động không tốt cho mô hình.
* **Hiệu quả và linh hoạt**: Trong khi tiền xử lý có thể làm ở ngoại tuyến (chẳng hạn bằng cách viết các đầu ra đã được xử lý vào tập tin lưu trên đĩa và sau đó xem xét lại các dữ liệu đã được tiền xử lý trong đường tin đầu vào), phương pháp này phát sinh thêm chi phí đọc-ghi dữ liệu. Tiền xử lý ngoại tuyến thường bất tiện nếu có các quyết định tiền xử lý cần xảy ra động. Thử nghiệm với một tuỳ chọn khác nhau sẽ yêu cầu tạo lại tập dữ liệu một lần nữa.
* **Giao diện mô hình phức tạp**: Các mô hình văn bản dễ hiểu hơn nhiều khi các đầu vào của nó là các văn bản thuần. Nó cũng gây khó hiểu một mô hình khi đầu vào yêu cầu các bước mã hoá gián tiếp bổ sung. Giảm độ phức tạp của quá trình tiền xử lý được đánh giá cao cho quá trình sửa lỗi mô hình, phục vụ và đánh giá.

Thêm vào đó, giao diện mô hình đơn giản hơn cũng tạo thêm sự thuận tiện để thử mô hình (chẳng hạn dùng để suy luận hay huấn luyện) trên các tập dữ liệu khác nhau, chưa được khám phá.

## Tiền xử lý dữ liệu với TF.Text

Sử dụng các API tiền xử lý văn bản của TF.Text, chúng ta có thể xây dựng một hàm tiền xử lý mà có thể chuyển đổi một tập dữ liệu văn bản của người dùng thành các đầu vào số nguyên của mô hình. Người dùng có thể đóng gói trực tiếp tiền xử lý như là một phần của mô hình để giảm bớt các vấn đề đã đề cập ở trên.

Hướng dẫn này sẽ trình bày cách sử dụng các hoạt động tiền xử lý TF.Text để chuyển đổi dữ liệu văn bản thành các đầu vào cho mô hình BERT và các đầu vào cho nhiệm vụ tiền huấn luyện mặt nạ ngôn ngữ được mô tả trong "*Masked LM and Masking Procedure*" của [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf). Quá trình liên quan đến token hoá văn bản thành các đơn vị từ con, loại bỏ nội dung để vừa khít kích thước và trích xuất nhãn cho nhiệm vụ mô hình hoá mặt nạ ngôn ngữ.

### Văn bản

Nhập các gói và thư viện cần thiết:

In [ ]:
!pip install -q -U tensorflow-text

In [ ]:
import tensorflow as tf
import tensorflow_text as text
import functools

Dữ liệu của chúng ta chứa hai đặc trưng văn bản và chúng ta có thể tạo một ví dụ `tf.data.Dataset`. Mục tiêu của chúng ta là tạo một hàm mà chúng ta có thể cung cấp `Dataset.map()` vào huấn luyện mô hình.

In [ ]:
examples = {
    "text_a": [
      b"Sponge bob Squarepants is an Avenger",
      b"Marvel Avengers"
    ],
    "text_b": [
     b"Barack Obama is the President.",
     b"President is the highest office"
  ],
}

dataset = tf.data.Dataset.from_tensor_slices(examples)
next(iter(dataset))

### Toeken hoá

Bước đầu tiên là chạy tiền xử lý chuỗi bất kỳ và token tập dữ liệu. Điều này có thể thực hiện bởi `text.BertTokenizer`, nó là một hàm `text.Splitter` token các câu thành các từ con hoặc các mảnh từ cho [mô hình BERT](https://github.com/google-research/bert) với một từ vựng được tạo ra từ [thuật toán Wordpiece](https://www.tensorflow.org/text/guide/subwords_tokenizer#optional_the_algorithm).

Từ vựng có thể là các điểm kiểm tra BERT được tạo trước đó, hoặc bạn có thể tạo một từ mới trên dữ liệu của bạn. Đối với các mục đích của ví dụ này, hãy tạo một từ vựng đồ chơi.

In [ ]:
_VOCAB = [
    # Special tokens
    b"[UNK]", b"[MASK]", b"[RANDOM]", b"[CLS]", b"[SEP]",
    # Suffixes
    b"##ack", b"##ama", b"##ger", b"##gers", b"##onge", b"##pants",  b"##uare",
    b"##vel", b"##ven", b"an", b"A", b"Bar", b"Hates", b"Mar", b"Ob",
    b"Patrick", b"President", b"Sp", b"Sq", b"bob", b"box", b"has", b"highest",
    b"is", b"office", b"the",
]

_START_TOKEN = _VOCAB.index(b"[CLS]")
_END_TOKEN = _VOCAB.index(b"[SEP]")
_MASK_TOKEN = _VOCAB.index(b"[MASK]")
_RANDOM_TOKEN = _VOCAB.index(b"[RANDOM]")
_UNK_TOKEN = _VOCAB.index(b"[UNK]")
_MAX_SEQ_LEN = 8
_MAX_PREDICTIONS_PER_BATCH = 5
 
_VOCAB_SIZE = len(_VOCAB)

lookup_table = tf.lookup.StaticVocabularyTable(
    tf.lookup.KeyValueTensorInitializer(
      keys=_VOCAB,
      key_dtype=tf.string,
      values=tf.range(
          tf.size(_VOCAB, out_type=tf.int64), dtype=tf.int64),
      value_dtype=tf.int64),
      num_oov_buckets=1
)

Xây dựng một `text.BertTokenizer` sử dụng từ vựng ở trên và token đầu vào văn bản thành một `RaggedTensor`.

In [ ]:
bert_tokenizer = text.BertTokenizer(lookup_table, token_out_type=tf.string)
bert_tokenizer.tokenize(examples["text_a"])

In [ ]:
bert_tokenizer.tokenize(examples["text_b"])

Đầu ra văn bản từ `text.BertTokenizer` cho phép chúng ta xem cách văn bản được token hoá, nhưng mô hình yêu câu các ID số nguyên. Chúng ta có thể đặt tham số `token_out_type` thành `tf.int64` để lấy các ID nguyên (là các chỉ số trong từ vựng).

In [ ]:
bert_tokenizer = text.BertTokenizer(lookup_table, token_out_type=tf.int64)
segment_a = bert_tokenizer.tokenize(examples["text_a"])
segment_a

In [ ]:
segment_b = bert_tokenizer.tokenize(examples["text_b"])
segment_b

`text.BertTokenizer` trả về một RaggedTensor với hình dạng `[batch, num_tokens, num_wordpieces]`. Bởi vì chúng ta không cần chiều `num_tokens` bổ sung cho trường hợp sử dụng hiện tại, chúng ta có thể gộp hai chiều cuối cùng để có được một `RaggedTensor` với hình dạng `[batch, num_wordpieces]`:

In [ ]:
segment_a = segment_a.merge_dims(-2, -1)
segment_a

In [ ]:
segment_b = segment_b.merge_dims(-2, -1)
segment_b